# Oanda: How to create a stop loss & take profit

In [12]:
from trader import ConTrader

In [14]:
trader = ConTrader("Oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [16]:
trader.get_most_recent()

In [18]:
trader.define_strategy()

In [20]:
current_price = trader.data[trader.instrument].iloc[-1]
current_price

1.13716

In [22]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                           sl_distance = 0.01)
order

{'id': '492',
 'time': '2025-05-06T21:36:55.302633018Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '491',
 'requestID': '97397627246140012',
 'type': 'ORDER_FILL',
 'orderID': '491',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.1373,
 'fullVWAP': 1.1373,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13687, 'liquidity': '500000'},
   {'price': 1.13686, 'liquidity': '2500000'},
   {'price': 1.13685, 'liquidity': '2000000'},
   {'price': 1.13684, 'liquidity': '5000000'},
   {'price': 1.13681, 'liquidity': '10000000'},
   {'price': 1.13678, 'liquidity': '10000000'}],
  'asks': [{'price': 1.1373, 'liquidity': '500000'},
   {'price': 1.13731, 'liquidity': '500000'},
   {'price': 1.13732, 'liquidity': '2000000'},
   {'price': 1.13733, 'liquidity': '2000000'},
   {'price': 1.13734, 'liquidity': '5000000'},
   {'price': 1.13736, 'liquidity': '10000000'},
   {'price

In [24]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '495',
 'time': '2025-05-06T21:36:56.721374895Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '494',
 'requestID': '61368830231612256',
 'type': 'ORDER_FILL',
 'orderID': '494',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13687,
 'fullVWAP': 1.13687,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13687, 'liquidity': '500000'},
   {'price': 1.13686, 'liquidity': '2500000'},
   {'price': 1.13685, 'liquidity': '2000000'},
   {'price': 1.13684, 'liquidity': '5000000'},
   {'price': 1.13681, 'liquidity': '10000000'},
   {'price': 1.13678, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13728, 'liquidity': '499900'},
   {'price': 1.13729, 'liquidity': '500000'},
   {'price': 1.1373, 'liquidity': '2000000'},
   {'price': 1.13731, 'liquidity': '2000000'},
   {'price': 1.13732, 'liquidity': '5000000'},
   {'price': 1.13734, 'liquidity': '10000000'},
   {'pr

# Adding Limit Orders to Trading Bot (Take profit & Stop Loss)

### Oanda: Contrader Class without Trake Profits & Stop Loss

In [28]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta, timezone # Need to import timezone seperatly
import time

In [30]:
class ConTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, window, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = [] 
        
        #*****************add strategy-specific attributes here******************
        self.window = window
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.now(timezone.utc).replace(tzinfo=None) # new (Python 3.12)
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.now(timezone.utc)) - self.last_bar < self.bar_length: # new (Python 3.12)
                break
            
    def start_trading(self, days, max_attempts = 5, wait = 20, wait_increase = 0): # NEW, erroe handling
        attempt = 0
        success = False 
        while True:
            try:
                self.get_most_recent(days)
                self.stream_data(self.instrument)
            except Exception as e:
                print(e, end = " | ")
            else:
                success = True
                break
            finally:
                attempt += 1
                print("Attempt: {}".format(attempt), end = 'n/')
                if success == False:
                    if attempt >= max_attempts:
                        print("Max_attempts reached!")
                        try: # for termination 
                            time.sleep(wait)
                            self.terminate_session(cause = "Unexpected Session Stop (too many errors).")
                        except Exception as e:
                            print(e, end = " | ")
                            print("Could not terminate the session.")
                        finally:
                            break
                    else:
                        time.sleep(wait)
                        wait += wait_increase
                        self.tick_data = pd.DataFrame()
                     
    def on_success(self, time, bid, ask):
        print(self.ticks, end = "\r", flush = True)
        
        recent_tick = pd.to_datetime(time)
        
        # define stop
        if self.ticks >= 100:
            self.terminate_session(cause = "Scheduled Session End.")
            return
        
        # collect and store tick data
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = pd.concat([self.tick_data, df]) 
        
        # if a time longer than the bar_lenght has elapsed between last full bar and the most recent tick
        if recent_tick - self.last_bar >= self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
            
    def resample_and_join(self):
        self.raw_data = pd.concat([self.raw_data, self.tick_data.resample(self.bar_length, 
                                                                          label="right").last().ffill().iloc[:-1]])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["returns"] = np.log(df[self.instrument] / df[self.instrument].shift())
        df["position"] = -np.sign(df.returns.rolling(self.window).mean())
        #***********************************************************************
        
        self.data = df.copy()
        
    def execute_trades(self):
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):  
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        print("\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}".format(time, units, price, pl, cumpl))
        print(100 * "-" + "\n")  
        
    def terminate_session(self, cause): # NEW
        self.stop_stream = True
        if self.position != 0:
            close_order = self.create_order(self.instrument, units = -self.position * self.units,
                                            suppress = True, ret = True) 
            self.report_trade(close_order, "GOING NEUTRAL")
            self.position = 0
        print(cause)

In [32]:
trader = ConTrader("oanda.cfg", "EUR_USD", "1min", window = 1, units = 100)

In [34]:
trader.start_trading(days = 5, max_attempts = 3, wait = 20, wait_increase = 0)

1
----------------------------------------------------------------------------------------------------
2025-05-06T21:37:07.810741195Z | GOING SHORT
2025-05-06T21:37:07.810741195Z | units = -100.0 | price = 1.13687 | P&L = 0.0 | Cum P&L = 0.0
----------------------------------------------------------------------------------------------------

20
----------------------------------------------------------------------------------------------------
2025-05-06T21:38:03.910268138Z | GOING LONG
2025-05-06T21:38:03.910268138Z | units = 200.0 | price = 1.13724 | P&L = -0.037 | Cum P&L = -0.037
----------------------------------------------------------------------------------------------------

38
----------------------------------------------------------------------------------------------------
2025-05-06T21:40:47.122050808Z | GOING SHORT
2025-05-06T21:40:47.122050808Z | units = -200.0 | price = 1.13693 | P&L = -0.031 | Cum P&L = -0.068
----------------------------------------------------------

In [38]:
trader.data

,EUR_USD,returns,position
2025-05-01 21:36:00+00:00,1.128920,NaN,NaN
2025-05-01 21:37:00+00:00,1.128880,-0.000035,1.0
2025-05-01 21:38:00+00:00,1.128880,0.000000,-0.0
2025-05-01 21:39:00+00:00,1.128880,0.000000,-0.0
2025-05-01 21:40:00+00:00,1.128900,0.000018,-1.0
...,...,...,...
2025-05-06 21:51:00+00:00,1.137300,0.000084,-1.0
2025-05-06 21:52:00+00:00,1.137325,0.000022,-1.0
2025-05-06 21:53:00+00:00,1.137325,0.000000,-0.0
2025-05-06 21:54:00+00:00,1.137300,-0.000022,1.0


In [40]:
trader.position

0

## Stop loss and take profit

In [43]:
trader

In [45]:
current_price = trader.data[trader.instrument].iloc[-1]
current_price

1.1373000000000002

In [47]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True, # open a long position
                           sl_distance = 0.01)
order

{'id': '514',
 'time': '2025-05-06T21:56:40.326554657Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '513',
 'requestID': '97397632212750961',
 'type': 'ORDER_FILL',
 'orderID': '513',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13743,
 'fullVWAP': 1.13743,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13711, 'liquidity': '500000'},
   {'price': 1.1371, 'liquidity': '2500000'},
   {'price': 1.13709, 'liquidity': '2000000'},
   {'price': 1.13708, 'liquidity': '5000000'},
   {'price': 1.13705, 'liquidity': '10000000'},
   {'price': 1.13702, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '500000'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pri

In [49]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True) # Closing position manually
order

{'id': '517',
 'time': '2025-05-06T21:56:41.089441606Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '516',
 'requestID': '79383233710443371',
 'type': 'ORDER_FILL',
 'orderID': '516',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13711,
 'fullVWAP': 1.13711,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13711, 'liquidity': '500000'},
   {'price': 1.1371, 'liquidity': '2500000'},
   {'price': 1.13709, 'liquidity': '2000000'},
   {'price': 1.13708, 'liquidity': '5000000'},
   {'price': 1.13705, 'liquidity': '10000000'},
   {'price': 1.13702, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '499900'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pr

#### Take Profit

In [52]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                            tp_price = 1.14) # Adjust price as needed.
order

{'id': '519',
 'time': '2025-05-06T21:56:41.982342034Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '519',
 'requestID': '61368835198358317',
 'type': 'MARKET_ORDER_REJECT',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'timeInForce': 'FOK',
 'positionFill': 'DEFAULT',
 'reason': 'CLIENT_ORDER',
 'takeProfitOnFill': {'price': 1.13999999999999, 'timeInForce': 'GTC'},
 'rejectReason': 'TAKE_PROFIT_ON_FILL_PRICE_PRECISION_EXCEEDED'}

In [54]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '521',
 'time': '2025-05-06T21:56:42.936480869Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '520',
 'requestID': '79383233714638724',
 'type': 'ORDER_FILL',
 'orderID': '520',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13711,
 'fullVWAP': 1.13711,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13711, 'liquidity': '499900'},
   {'price': 1.1371, 'liquidity': '2500000'},
   {'price': 1.13709, 'liquidity': '2000000'},
   {'price': 1.13708, 'liquidity': '5000000'},
   {'price': 1.13705, 'liquidity': '10000000'},
   {'price': 1.13702, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '499900'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pr

#### Combining SL and TP (Stop Loss and Take Profit)

In [57]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                            sl_distance = 0.01, tp_price = 1.11)
order

{'id': '523',
 'time': '2025-05-06T21:56:44.059876163Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '522',
 'requestID': '79383233723027937',
 'type': 'ORDER_FILL',
 'orderID': '522',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13743,
 'fullVWAP': 1.13743,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13711, 'liquidity': '499800'},
   {'price': 1.1371, 'liquidity': '2500000'},
   {'price': 1.13709, 'liquidity': '2000000'},
   {'price': 1.13708, 'liquidity': '5000000'},
   {'price': 1.13705, 'liquidity': '10000000'},
   {'price': 1.13702, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '499900'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pri

In [59]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '525',
 'time': '2025-05-06T21:56:45.808866400Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '524',
 'requestID': '61368835215137739',
 'type': 'ORDER_FILL',
 'orderID': '524',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13711,
 'fullVWAP': 1.13711,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13711, 'liquidity': '499800'},
   {'price': 1.1371, 'liquidity': '2500000'},
   {'price': 1.13709, 'liquidity': '2000000'},
   {'price': 1.13708, 'liquidity': '5000000'},
   {'price': 1.13705, 'liquidity': '10000000'},
   {'price': 1.13702, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '499800'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pr

#### Trailing Stop Loss

Trails the price takes a stop loss based on tsl_distance

In [63]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True,
                            tsl_distance = 0.01)
order

{'id': '527',
 'time': '2025-05-06T21:56:47.217278971Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '526',
 'requestID': '61368835223527205',
 'type': 'ORDER_FILL',
 'orderID': '526',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13743,
 'fullVWAP': 1.13743,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13711, 'liquidity': '499700'},
   {'price': 1.1371, 'liquidity': '2500000'},
   {'price': 1.13709, 'liquidity': '2000000'},
   {'price': 1.13708, 'liquidity': '5000000'},
   {'price': 1.13705, 'liquidity': '10000000'},
   {'price': 1.13702, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '499800'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pri

In [65]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '529',
 'time': '2025-05-06T21:56:49.404083642Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '528',
 'requestID': '61368835231917096',
 'type': 'ORDER_FILL',
 'orderID': '528',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13713,
 'fullVWAP': 1.13713,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13713, 'liquidity': '499700'},
   {'price': 1.13712, 'liquidity': '2500000'},
   {'price': 1.13711, 'liquidity': '2000000'},
   {'price': 1.1371, 'liquidity': '5000000'},
   {'price': 1.13707, 'liquidity': '10000000'},
   {'price': 1.13704, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '499700'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pr

# Stop loss and Take Profit Orders - Pitfalls

In [68]:
trader

In [70]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True, # Order Cancelled example, distance is to small do to the pip.
                            sl_distance = 0.0001)
order

{'id': '531',
 'time': '2025-05-06T21:56:51.603199163Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '530',
 'requestID': '79383233752392195',
 'type': 'ORDER_FILL',
 'orderID': '530',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13743,
 'fullVWAP': 1.13743,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13713, 'liquidity': '499600'},
   {'price': 1.13712, 'liquidity': '2500000'},
   {'price': 1.13711, 'liquidity': '2000000'},
   {'price': 1.1371, 'liquidity': '5000000'},
   {'price': 1.13707, 'liquidity': '10000000'},
   {'price': 1.13704, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '499700'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pri

In [72]:
order["price"] # order is incorrect the sl_distance was to small.

1.13743

Another example of common mistakes when setting stop or limit orders is to many decimals. Below is an example of it.

In [75]:
order = trader.create_order(trader.instrument, trader.units, suppress = True, ret = True, # Order Cancelled example, distance is to small do to the pip.
                            sl_distance = 0.00014962)
order

{'id': '532',
 'time': '2025-05-06T21:56:53.838822938Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '532',
 'requestID': '97397632267283417',
 'type': 'MARKET_ORDER_REJECT',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'timeInForce': 'FOK',
 'positionFill': 'DEFAULT',
 'reason': 'CLIENT_ORDER',
 'stopLossOnFill': {'distance': '0.00014962', 'timeInForce': 'GTC'},
 'rejectReason': 'STOP_LOSS_ON_FILL_DISTANCE_PRECISION_EXCEEDED'}

Another common mistakes is setting the stop or limit orders to small or great based on price. 

# Setting SL Distance and TP Price in real-time

In [79]:
trader

In [81]:
current_price = trader.get_prices(trader.instrument)[2]
current_price

1.13752

In [83]:
sl_perc = 0.01 # Stop loss if price moves in wrong direction, stop at 1%

In [85]:
sl_dist = round(current_price * sl_perc, 4) # remember rouding to 4 decimal places
sl_dist

0.0114

In [87]:
current_price - sl_dist # stop price for long(buy)

1.12612

In [89]:
current_price + sl_dist # stop price for a short

1.1489200000000002

#### Setting take profit price

In [92]:
from decimal import Decimal, ROUND_HALF_UP, getcontext

In [94]:
getcontext().prec = 6

In [96]:
target_position = 1 

In [98]:
tp_perc = Decimal("0.03") # 3% in price difference

In [100]:
current_price_decimal = Decimal(str(current_price))

In [102]:
if target_position == 1:
    tp_price = current_price_decimal * (Decimal("1.0") + tp_perc)
elif target_position == -1:
    tp_price = current_price_decimal * (Decimal("1.0") - tp_perc)

In [104]:
tp_price = Decimal(tp_price).quantize(Decimal("0.0001"), rounding=ROUND_HALF_UP)

In [106]:
tp_price = float(format(tp_price, '.5f'))

In [108]:
order = trader.create_order(
    trader.instrument,
    trader.units,
    suppress=True,
    ret=True,
    sl_distance=sl_dist,
    tp_price=tp_price
)

order

{'id': '533',
 'time': '2025-05-06T21:57:06.069821183Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '533',
 'requestID': '79383233815315452',
 'type': 'MARKET_ORDER_REJECT',
 'instrument': 'EUR_USD',
 'units': '100.0',
 'timeInForce': 'FOK',
 'positionFill': 'DEFAULT',
 'reason': 'CLIENT_ORDER',
 'takeProfitOnFill': {'price': 1.17169999999999, 'timeInForce': 'GTC'},
 'stopLossOnFill': {'distance': '0.0114', 'timeInForce': 'GTC'},
 'rejectReason': 'TAKE_PROFIT_ON_FILL_PRICE_PRECISION_EXCEEDED'}

In [110]:
order = trader.create_order(trader.instrument, -trader.units, suppress = True, ret = True)
order

{'id': '535',
 'time': '2025-05-06T21:57:06.656984677Z',
 'userID': 29655670,
 'accountID': '101-001-29655670-001',
 'batchID': '534',
 'requestID': '97397632321816070',
 'type': 'ORDER_FILL',
 'orderID': '534',
 'instrument': 'EUR_USD',
 'units': '-100.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.13716,
 'fullVWAP': 1.13716,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.13716, 'liquidity': '500000'},
   {'price': 1.13715, 'liquidity': '2500000'},
   {'price': 1.13714, 'liquidity': '2000000'},
   {'price': 1.13713, 'liquidity': '5000000'},
   {'price': 1.1371, 'liquidity': '10000000'},
   {'price': 1.13707, 'liquidity': '10000000'}],
  'asks': [{'price': 1.13743, 'liquidity': '500000'},
   {'price': 1.13744, 'liquidity': '500000'},
   {'price': 1.13745, 'liquidity': '2000000'},
   {'price': 1.13746, 'liquidity': '2000000'},
   {'price': 1.13747, 'liquidity': '5000000'},
   {'price': 1.13749, 'liquidity': '10000000'},
   {'pr

# Check for SL / TP events

In [115]:
trader.position = 1
trader.position

1

In [119]:
exp_position = round(trader.position*trader.units, 0)
exp_position

100

In [125]:
positions = trader.get_positions()
positions

[{'instrument': 'EUR_USD',
  'pl': '223.422',
  'unrealizedPL': '0.393',
  'marginUsed': '2.2663',
  'resettablePL': '223.422',
  'financing': '-253.6959',
  'commission': '0.0',
  'guaranteedExecutionFees': '0.0',
  'long': {'units': '0.0',
   'pl': '271.166',
   'unrealizedPL': '0.0',
   'resettablePL': '271.166',
   'financing': '-253.8575',
   'guaranteedExecutionFees': '0.0'},
  'short': {'units': '-100.0',
   'averagePrice': 1.13716,
   'tradeIDs': ['535'],
   'pl': '-47.744',
   'unrealizedPL': '0.393',
   'resettablePL': '-47.744',
   'financing': '0.1616',
   'guaranteedExecutionFees': '0.0'}}]

In [127]:
actual_position = 0
for pos in positions:
    if pos["instrument"] == trader.instrument:
        actual_position = round(float(pos["long"]["units"]) + float(pos["short"]["units"]), 0)
actual_position

-100.0

In [129]:
if actual_position != exp_position:
    trader.position = actual_position / trader.units
    print("Go Neutral and Stop Trading Session")

Go Neutral and Stop Trading Session


In [131]:
trader.position

-1.0